Universidad del Valle de Guatemala  
Departamento de Ciencias de la Computación  
Minería de Datos - sección 30  

Cristian Túnchez - 231359  
Javier Linares - 231135

---

# Proyecto 1: Análisis Exploratorio y Clustering

## 1. Consolidación de Datos de Divorcios 2010-2020

In [1]:
# Importar librerías necesarias
import pyreadstat
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas exitosamente")

Librerías cargadas exitosamente


In [18]:
# Paso 1: Explorar la estructura de cada archivo

years = range(2010, 2021)
file_info = {}

for year in years:
    filename = f'datasets/INE_Divorcios_{year}.sav'
    df, meta = pyreadstat.read_sav(filename)
    file_info[year] = {
        'shape': df.shape,
        'columns': df.columns.tolist()
    }
    print(f'{year}: {df.shape[0]} registros, {df.shape[1]} columnas')

print(f'\nTotal de archivos cargados: {len(file_info)}')

2010: 3645 registros, 19 columnas
2011: 4344 registros, 19 columnas
2012: 5157 registros, 18 columnas
2013: 5542 registros, 18 columnas
2014: 5575 registros, 18 columnas
2015: 5726 registros, 19 columnas
2016: 5665 registros, 19 columnas
2017: 5808 registros, 19 columnas
2018: 6255 registros, 19 columnas
2019: 8203 registros, 19 columnas
2020: 4074 registros, 19 columnas

Total de archivos cargados: 11


In [10]:
# Paso 2: Identificar años con columnas faltantes
# Verificamos cuáles años NO tienen la columna de año de ocurrencia

print("Verificación de columna AÑOOCU:")
for year in years:
    cols_lower = [c.lower() for c in file_info[year]['columns']]
    has_anoocu = 'añoocu' in cols_lower
    symbol = '✓' if has_anoocu else '✗ FALTA'
    print(f'  {year}: {symbol}')

Verificación de columna AÑOOCU:
  2010: ✓
  2011: ✓
  2012: ✗ FALTA
  2013: ✗ FALTA
  2014: ✗ FALTA
  2015: ✓
  2016: ✓
  2017: ✓
  2018: ✓
  2019: ✓
  2020: ✓


### Paso 3: Definir mapeo de estandarización de columnas

**Decisiones tomadas:**
- **Esquema objetivo**: camelCase para nombres de columnas
- **Estrategia**: Convertir todos los nombres a minúsculas primero, luego aplicar mapeo único
- **Columnas identificadas como equivalentes**:
  - `grethom/gethom/puehom/pperhom` → `grupoEtnicoHombre`
  - `gretmuj/getmuj/puemuj/ppermuj` → `grupoEtnicoMujer`
  - `escohom/eschom` → `escolaridadHombre`
  - `escomuj/escmuj` → `escolaridadMujer`
  - `ocupahom/ocuhom/ciuohom` → `ocupacionHombre`
  - `ocupamuj/ocumuj/ciuomuj` → `ocupacionMujer`

In [12]:
# Mapeo único para todas las variaciones de nombres de columnas
# Se convertirán todas las columnas a minúsculas antes de aplicar este mapeo
column_mapping = {
    'depreg': 'departamentoRegistro',
    'mupreg': 'municipioRegistro',
    'mesreg': 'mesRegistro',
    'añoreg': 'anioRegistro',
    'diaocu': 'diaOcurrencia',
    'mesocu': 'mesOcurrencia',
    'añoocu': 'anioOcurrencia',
    'depocu': 'departamentoOcurrencia',
    'mupocu': 'municipioOcurrencia',
    'edadhom': 'edadHombre',
    'edadmuj': 'edadMujer',
    'grethom': 'grupoEtnicoHombre',
    'gethom': 'grupoEtnicoHombre',
    'puehom': 'grupoEtnicoHombre',
    'pperhom': 'grupoEtnicoHombre',
    'gretmuj': 'grupoEtnicoMujer',
    'getmuj': 'grupoEtnicoMujer',
    'puemuj': 'grupoEtnicoMujer',
    'ppermuj': 'grupoEtnicoMujer',
    'nachom': 'nacionalidadHombre',
    'nacmuj': 'nacionalidadMujer',
    'escohom': 'escolaridadHombre',
    'eschom': 'escolaridadHombre',
    'escomuj': 'escolaridadMujer',
    'escmuj': 'escolaridadMujer',
    'ocupahom': 'ocupacionHombre',
    'ocuhom': 'ocupacionHombre',
    'ciuohom': 'ocupacionHombre',
    'ocupamuj': 'ocupacionMujer',
    'ocumuj': 'ocupacionMujer',
    'ciuomuj': 'ocupacionMujer'
}

print(f"Mapeo de columnas definido ({len(set(column_mapping.values()))} columnas únicas).")

Mapeo de columnas definido (19 columnas únicas).


### Paso 4: Cargar, estandarizar y consolidar todos los archivos

**Proceso:**
1. Cargar cada archivo .sav
2. Convertir nombres de columnas a minúsculas
3. Crear columna `añoocu` para los años 2012-2014 (usando el año del archivo)
4. Aplicar el mapeo único de columnas
5. Concatenar todos los DataFrames en uno solo

In [13]:
# Función para procesar un archivo según su año
def procesar_archivo(year):
    """Carga y procesa un archivo .sav según su año"""
    filename = f'datasets/INE_Divorcios_{year}.sav'
    df, meta = pyreadstat.read_sav(filename)
    
    # Convertir todos los nombres de columnas a minúsculas
    df.columns = df.columns.str.lower()
    
    # Verificar si falta la columna añoocu (años 2012-2014)
    if 'añoocu' not in df.columns:
        # Crear la columna usando el año del archivo
        df['añoocu'] = year
    
    # Aplicar el mapeo de columnas
    df = df.rename(columns=column_mapping)
    
    return df

# Procesar todos los archivos
print("Procesando archivos...")
dataframes = []

for year in years:
    df = procesar_archivo(year)
    dataframes.append(df)
    print(f'- {year}: {df.shape[0]:,} registros, {df.shape[1]} columnas')

print(f'\nTotal archivos procesados: {len(dataframes)}')

Procesando archivos...
- 2010: 3,645 registros, 19 columnas
- 2011: 4,344 registros, 19 columnas
- 2012: 5,157 registros, 19 columnas
- 2013: 5,542 registros, 19 columnas
- 2014: 5,575 registros, 19 columnas
- 2015: 5,726 registros, 19 columnas
- 2016: 5,665 registros, 19 columnas
- 2017: 5,808 registros, 19 columnas
- 2018: 6,255 registros, 19 columnas
- 2019: 8,203 registros, 19 columnas
- 2020: 4,074 registros, 19 columnas

Total archivos procesados: 11


In [17]:
# Paso 5: Consolidar todos los DataFrames en uno solo
df_consolidado = pd.concat(dataframes, ignore_index=True)

print("="*60)
print("DATASET FINAL - DIVORCIOS GUATEMALA 2010-2020")
print("="*60)
print(f"\nDimensiones: {df_consolidado.shape[0]:,} filas × {df_consolidado.shape[1]} columnas")
print("Período: 2010-2020")
print(f"Memoria utilizada: {df_consolidado.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

DATASET FINAL - DIVORCIOS GUATEMALA 2010-2020

Dimensiones: 59,994 filas × 19 columnas
Período: 2010-2020
Memoria utilizada: 18.42 MB


### Paso 6: Visualización del dataset consolidado

In [15]:
# Lista de columnas estandarizadas
print("COLUMNAS DEL DATASET CONSOLIDADO:")
print("-" * 60)
for i, col in enumerate(df_consolidado.columns, 1):
    print(f"{i:2}. {col}")
print("-" * 60)
print(f"Total: {len(df_consolidado.columns)} columnas")

COLUMNAS DEL DATASET CONSOLIDADO:
------------------------------------------------------------
 1. departamentoRegistro
 2. municipioRegistro
 3. mesRegistro
 4. anioRegistro
 5. diaOcurrencia
 6. mesOcurrencia
 7. anioOcurrencia
 8. departamentoOcurrencia
 9. municipioOcurrencia
10. edadHombre
11. edadMujer
12. grupoEtnicoHombre
13. grupoEtnicoMujer
14. nacionalidadHombre
15. nacionalidadMujer
16. escolaridadHombre
17. escolaridadMujer
18. ocupacionHombre
19. ocupacionMujer
------------------------------------------------------------
Total: 19 columnas


In [16]:
# Primeras 5 filas del dataset consolidado
print("\nPRIMERAS 5 FILAS DEL DATASET:")
print("=" * 100)
df_consolidado.head()


PRIMERAS 5 FILAS DEL DATASET:


,departamentoRegistro,municipioRegistro,mesRegistro,anioRegistro,diaOcurrencia,mesOcurrencia,anioOcurrencia,departamentoOcurrencia,municipioOcurrencia,edadHombre,edadMujer,grupoEtnicoHombre,grupoEtnicoMujer,nacionalidadHombre,nacionalidadMujer,escolaridadHombre,escolaridadMujer,ocupacionHombre,ocupacionMujer
0,1.0,0101,8.0,2010.0,4.0,5.0,2010.0,1.0,0101,41.0,35.0,9.0,9.0,320.0,320.0,9.0,9.0,1120.0,1120.0
1,1.0,0101,11.0,2010.0,6.0,10.0,2010.0,1.0,0101,999.0,999.0,9.0,9.0,320.0,320.0,9.0,9.0,2161.0,1219.0
2,1.0,0101,3.0,2010.0,16.0,2.0,2010.0,1.0,0101,42.0,31.0,9.0,9.0,320.0,320.0,4.0,9.0,2166.0,1219.0
3,1.0,0101,10.0,2010.0,25.0,8.0,2010.0,1.0,0101,47.0,36.0,9.0,9.0,320.0,320.0,1.0,1.0,5221.0,1219.0
4,1.0,0101,8.0,2010.0,20.0,5.0,2010.0,1.0,0101,999.0,999.0,9.0,9.0,840.0,484.0,9.0,9.0,9712.0,1219.0
